In [14]:
# !pip install nlp
# !pip install aiohttp
# !pip install bert_score
# !pip install git+https://github.com/google-research/bleurt.git
# !pip install -U transformers 
# !pip install mosestokenizer


from pathlib import Path

import pandas as pd
import asyncio
import aiohttp
import json
from aiohttp import ClientConnectorError, ClientSession
from nlp import load_metric
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [15]:
data_path = Path("../data")
assert data_path.exists()
datagen_path = Path("../data/datageneration")
assert datagen_path.exists()
keys_path = Path("key.json")
assert keys_path.exists()

In [16]:
data = pd.read_json(datagen_path/"SentencesUsingReasons.json")
# data.sample(10).to_json(data_path/"prompts/ParaphrasedIntents.json", indent=2, index=False, orient="table")
data = data.dropna()
data

,Structure,Sentence,Input Sentence,Intent,Reason
0,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I need to know the last transaction
1,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to check my transactions
2,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,To check my balance
3,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to see if my friend used it
4,"FormalWithGreetings, ReasonFirst","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to know what was the last transaction o...
...,...,...,...,...,...
5945,"FormalWithGreetings, ReasonFirst","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to check what was my last transaction
5946,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to check if I have been scammed
5947,"WithReason, FormalWithGreetings","Hello, Good Morning, I wanted to know what wa...",What was the last debit transaction on my Debi...,Debit Transaction,I want to check if it's my card
5948,"FormalWithGreetings, WithReason, ReasonFirst",Hello I want to know what was the last transa...,What was the last debit transaction on my Debi...,Debit Transaction,I want to know what was the last transaction done


In [17]:
def prompt_generation(prompt_path):
    prompt = json.load(open(prompt_path))
    prompt = list(prompt)
    output = []
    for ele in prompt:
        for key, value in ele.items():
            output.append(f"{key} : {value}\n")
    return "".join(output)


prompt_path = data_path / "prompts" / "ParaphrasedIntents.json"
print(prompt_generation(prompt_path))

Sentence :  Hello, I want to know about my card status so can you please tell me?
Intent : Card Status
Context : Bank
Paraphrase : Hello, Can you tell me about the card status? It would be really nice if I can get this information
Sentence :  Hello, I wanted to know my payment due
Intent : Payment Amount Due
Context : Bank
Paraphrase : I want information about payment amount due, who do I contact about this?
Sentence :  HELLO, I want to register to security PIN because I want to use it for online transactions.
Intent : e-PIN Registration
Context : Bank
Paraphrase : I want to use e-pin for online transactions so I would like to know about how to register for security pin
Sentence :  I want to know the status of my covered card/debit card because i am going to travel.
Intent : Card Status
Context : Bank
Paraphrase : I am going to travel. Can you tell me about eh covered card status
Sentence :  I would like to donate to Zakat because I want to be good.
Intent : Donation
Context : Bank
Par

In [18]:
print(prompt_path.open("r").read())

[
    {
      "Sentence":" Hello, I want to know about my card status so can you please tell me?",
      "Intent":"Card Status",
      "Context" :"Bank",
      "Paraphrase" : "Hello, Can you tell me about the card status? It would be really nice if I can get this information"
      
    },
    {
      "Sentence":" Hello, I wanted to know my payment due",
      "Intent":"Payment Amount Due",
      "Context" :"Bank",
      "Paraphrase" : "I want information about payment amount due, who do I contact about this?"
      
    },
    {
      "Sentence":" HELLO, I want to register to security PIN because I want to use it for online transactions.",
      "Intent":"e-PIN Registration",
      "Context" :"Bank",
      "Paraphrase" : "I want to use e-pin for online transactions so I would like to know about how to register for security pin"
      
    },
    {
      "Sentence":" I want to know the status of my covered card\/debit card because i am going to travel.",
      "Intent":"Card Status",
 

In [19]:
async def make_requests_completions(query_list, prompt) -> None:
    async with ClientSession() as session:
        tasks = []
        url = "https://api.openai.com/v1/engines/curie/completions"
        for query in query_list:
            tasks.append(
                fetch_GPT3_completion_response(
                    url=url, session=session, prompt=prompt, query=query,
                )
            )
        results = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]
    return results

In [20]:
async def fetch_GPT3_completion_response(
    url: str, session: ClientSession, prompt, query
) -> tuple:
    headers = {
        "Content-Type": "application/json",
        "Authorization": f'Bearer {json.load(keys_path.open("r"))["api_key"]}',
    }
    try:
        prompt += query["Prompt"]
        data = json.dumps(
            {
                "prompt": prompt,
                "max_tokens": 50,
                "temperature": 0.55,
                "frequency_penalty": 0.80,
                "presence_penalty": 0.80,
                "stop": "\nSentence :",
                #                 "echo": True
            }
        )
        resp = await session.request(method="POST", url=url, headers=headers, data=data)
    except ClientConnectorError:
        return ("error", row.MessageId, query_intent)
    r = await resp.json()
    try:
        output = {
            "Structure": query["Structure"],
            "Sentence": query["Sentence"],
            "Input Sentence": query["Input Sentence"],
            "Intent": query["Intent"],
            "Reason": query["Reason"],
            "Paraphrased" : r["choices"][0]["text"].replace("Paraphrase : ", "")
        }
        return output
    except:

        return {
            "Structure": "NA",
            "Sentence": "NA",
        }

In [21]:
results = await make_requests_completions(
    query_list=[
        {
            "Prompt" : f"""Sentence : {row["Sentence"]}\nIntent : {row['Intent']}\n""",
#             "Input Sentence" : row["Input Sentence"],
#             "Intent" : row["Intent"],
#             "Reason" : row["Reason"],
#             "Structure" : row["Structure"],
#             "Sentence": row["Sentence"]
        }
        for _, row in data.iterrows()
    ],
    prompt=prompt_generation(prompt_path),
)

In [22]:
results

[{'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
 {'Structure': 'NA', 'Sentence': 'NA'},
